In [5]:
from pymongo import MongoClient
from pprint import pprint


In [6]:
import numpy as np
import pandas as pd
import requests

In [110]:
import datetime as dt

In [7]:
client = MongoClient()

In [8]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [9]:
client.collection_names

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'collection_names')

In [10]:
db = client.twitch

In [11]:
db.collection_names()

/var/folders/xl/ftndz_9j3hn5ybffb6t5q42m0000gn/T/ipykernel_41651/1579878703.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  db.collection_names()


['top_games', 'top_streams']

In [12]:
client.list_database_names()

['admin', 'books', 'config', 'events', 'local', 'outings']

# Creating twitch db.

I'm going to create a new database "Twitch" for this information. For now, I am just going to do one steam API pull for top streams, and see insert it into my database

In [12]:
db = client.twitch

In [13]:
## setting up twitch api permissions - taken from twitch_prelim notebook
# sensitive information removed, added to config file

client_id = 
client_secret = 

access_code = requests.post('https://id.twitch.tv/oauth2/token?client_id='+client_id+\
                            '&client_secret='+client_secret+\
                            '&grant_type=client_credentials')

access_token = access_code.json()['access_token']


headers = {
    'Client-ID' : client_id,
    'Authorization' : 'Bearer '+access_token
}

top_streams_pull = requests.get('https://api.twitch.tv/helix/streams?first=100', headers=headers)


In [20]:
top_streams_pull = requests.get('https://api.twitch.tv/helix/streams?first=100', headers=headers)
top_streams_pull

<Response [200]>

In [21]:
text = top_streams_pull.json()['data']
len(text)

100

In [22]:
top_streams = db.top_streams

In [23]:
db.top_streams.insert_many(text)

In [18]:
db.list_collection_names()

['top_games', 'top_streams']

In [19]:
top_streams.count_documents({}) #matches api pull

200

In [25]:
list(top_streams.find().limit(3))

[{'_id': ObjectId('60f9f81a6bbad902a94d2b83'),
  'id': '42721498588',
  'user_id': '489155160',
  'user_login': 'ranboolive',
  'user_name': 'RanbooLive',
  'game_id': '510694',
  'game_name': 'Observation',
  'type': 'live',
  'title': 'COOL SPACE GAME :D (Giveaway in ranmail channel right now!) FLASHING LIGHTS SOMETIMES BTW',
  'viewer_count': 55758,
  'started_at': '2021-07-22T22:03:36Z',
  'language': 'en',
  'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_ranboolive-{width}x{height}.jpg',
  'tag_ids': ['6ea6bca4-4712-4ab9-a906-e3336a9d8039'],
  'is_mature': False},
 {'_id': ObjectId('60f9f81a6bbad902a94d2b84'),
  'id': '42946210749',
  'user_id': '207813352',
  'user_login': 'hasanabi',
  'user_name': 'HasanAbi',
  'game_id': '509658',
  'game_name': 'Just Chatting',
  'type': 'live',
  'title': 'TW// MURDER REACTS W/ VALKYRAE !duo',
  'viewer_count': 43709,
  'started_at': '2021-07-22T18:08:27Z',
  'language': 'en',
  'thumbnail_url': 'https://static-cdn.jtv

In [26]:
top_games = db.top_games

In [50]:
top_games_pull = (requests.get('https://api.twitch.tv/helix/games/top?first=100', headers=headers)).json()['data']

In [53]:
id_list = []
for games in top_games_pull:
    id_list.append(games['id'])

In [55]:
for games in id_list:
    text = (requests.get('https://api.twitch.tv/helix/streams?first=100&game_id='+games, headers=headers)).json()['data']
    top_games.insert_many(text)

In [27]:
top_games.count_documents({})

8294

In [28]:
pd.DataFrame(list(top_games.find({'game_name':'Fortnite'},{'game_name','viewer_count'}).limit(3)))

,_id,game_name,viewer_count
0,60f9c5ebec884dc92537f250,Fortnite,14568
1,60f9c5ebec884dc92537f251,Fortnite,4171
2,60f9c5ebec884dc92537f252,Fortnite,3703


In [82]:
db.list_collection_names()

['continuous_streams', 'continuous_games', 'top_games', 'top_streams']

In [45]:
#list of uniuqe titles in continuously pulling db
list(db.continuous_games.find({},{'game_name':1,'_id':0}).distinct('game_name'))[:10]

['ASMR',
 'Apex Legends',
 'Art',
 'Axie Infinity',
 'Black Desert Online',
 'Call Of Duty: Modern Warfare',
 'Call of Duty: Black Ops 4',
 'Call of Duty: Black Ops Cold War',
 'Call of Duty: Mobile',
 'Call of Duty: Warzone']

In [54]:
fortnite = list(db.continuous_games.find({'game_name':'Fortnite'}))

In [58]:
fort_df = pd.DataFrame(fortnite)

In [59]:
fort_df.viewer_count.sum()

277633

In [60]:
fort_df

,_id,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,is_mature
0,60f9f87472c271762272d800,42721303580,233300375,clix,Clix,33214,Fortnite,live,High Kill Solo Arena | !sub !discord | NRG Clix,21554,2021-07-22T21:42:29Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],False
1,60f9f87472c271762272d801,42948024941,245226810,jelty,Jelty,33214,Fortnite,live,🤯EVENTO DE PRUEBA BUGHA ROYAL🤯 !bg || !exitlag...,18466,2021-07-22T20:38:11Z,es,https://static-cdn.jtvnw.net/previews-ttv/live...,"[d4bb9c58-2141-4881-bcdc-3fe0505457d1, c2839af...",False
2,60f9f87472c271762272d802,42719498604,82524912,bugha,Bugha,33214,Fortnite,live,"Squads w/ JuJu, K1, and Kurt !CodeBugha !skin ...",9271,2021-07-22T18:51:56Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[24cc3843-e288-43f9-9c35-50fe1571c341, 6ea6bca...",False
3,60f9f87472c271762272d803,42719993852,246450563,stableronaldo_,StableRonaldo_,33214,Fortnite,live,Bugha Test Event w/ ZORQ & ILLEST !talentshow,5927,2021-07-22T19:31:57Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],False
4,60f9f87472c271762272d804,42720401884,185619753,ops1x,ops1x,33214,Fortnite,live,"EU $3,000 BFC !bracket (1 year stream 36/365) ...",5076,2021-07-22T20:07:52Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,60fa14940ce3259904e7f6df,42946021709,635335361,sneepgg,SneepGG,33214,Fortnite,live,Arena 2nd 246K WORLDWIDE! | 189/365 Daily Str...,71,2021-07-22T17:54:16Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],False
292,60fa14940ce3259904e7f6e0,42949097725,402013783,dadplayer1,DadPlayer1,33214,Fortnite,live,ARENINHA COM KIDPLAYER !CAFE ! !KWAI !subgrati...,70,2021-07-22T22:03:24Z,pt,https://static-cdn.jtvnw.net/previews-ttv/live...,[39ee8140-901a-4762-bfca-8260dea1310f],False
293,60fa14940ce3259904e7f6e1,42950038813,42265041,coffee_senpai,Coffee_Senpai,33214,Fortnite,live,Extendible 4hr - Platicando mientras jugamos f...,70,2021-07-22T23:13:15Z,es,https://static-cdn.jtvnw.net/previews-ttv/live...,[d4bb9c58-2141-4881-bcdc-3fe0505457d1],False
294,60fa14940ce3259904e7f6e2,42721562076,597044009,astralsq,Astralsq,33214,Fortnite,live,2v2 vs sub w/Scott 7-0=50€ Live 12H 5Sub=+1H ...,70,2021-07-22T22:09:56Z,fr,https://static-cdn.jtvnw.net/previews-ttv/live...,[6f655045-9989-4ef7-8f85-1edcec42d648],False


In [62]:
df = pd.read_csv('streams.csv')

In [63]:
df

,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,is_mature
0,42700834796,44332309,sapnap,Sapnap,27471,Minecraft,live,Meeting An Old Friend... (lore),146374,2021-07-20T20:33:29Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,['6ea6bca4-4712-4ab9-a906-e3336a9d8039'],False
1,42914145581,26261471,asmongold,Asmongold,493597,New World,live,**DROPS** ENABLED--NEW WORLD BEGINS TODAY--DOI...,115977,2021-07-20T14:57:34Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"['c2542d6d-cd10-4532-919b-3d19f30a768b', '6ea6...",False
2,42917077229,83232866,ibai,ibai,509658,Just Chatting,live,"COSCU, IBAI Y NATHY PELUSO JUNTOS | EL TRIDENT...",106821,2021-07-20T19:16:07Z,es,https://static-cdn.jtvnw.net/previews-ttv/live...,['d4bb9c58-2141-4881-bcdc-3fe0505457d1'],False
3,42916885741,44445592,pokimane,pokimane,263490,Rust,live,BIGGEST & BEST DROPS ✔️ HIGHEST KDA ✔️ BEST LO...,47997,2021-07-20T19:01:57Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"['6ea6bca4-4712-4ab9-a906-e3336a9d8039', 'c254...",False
4,42914102893,36769016,timthetatman,TimTheTatman,493597,New World,live,*DROPS ENABLED* checking out new world,46764,2021-07-20T14:52:43Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"['e6bb8b34-4c28-4b5f-94ed-12c1ebf2d0e4', '6ea6...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,42913303965,63602976,skipnho,SkipNhO,493597,New World,live,Concentração de SparTano🟢Marauders🟢 SORTEIO DE...,5623,2021-07-20T13:12:14Z,pt,https://static-cdn.jtvnw.net/previews-ttv/live...,"['39ee8140-901a-4762-bfca-8260dea1310f', 'c254...",False
96,42699242188,177249859,pqueen,pqueen,493069,Totally Accurate Battle Simulator,live,Can Sungur ile Coop Geceler - instagram l pque...,5599,2021-07-20T18:17:46Z,tr,https://static-cdn.jtvnw.net/previews-ttv/live...,['f08d5873-f0c7-4912-94ba-a41933b4c141'],False
97,42697004684,219431490,jlamaru,JLAmaru,493597,New World,live,JL Amaru - On Test un Nouveau jeu OpenWorld ?!...,5541,2021-07-20T14:33:39Z,fr,https://static-cdn.jtvnw.net/previews-ttv/live...,"['6f655045-9989-4ef7-8f85-1edcec42d648', '2a14...",False
98,42700773708,217965779,punz,Punz,27471,Minecraft,live,MOUSEPADS DROP TODAY !mousepad | !gfuel !merch...,5509,2021-07-20T20:27:35Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,['6ea6bca4-4712-4ab9-a906-e3336a9d8039'],False


In [64]:
given = 'Fortnite'

In [68]:
temp = df[df.game_name == given]

In [71]:
temp.started_at = pd.to_datetime(temp.started_at)

/Users/michaelharnett/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [73]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 9 to 62
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   id             3 non-null      int64              
 1   user_id        3 non-null      int64              
 2   user_login     3 non-null      object             
 3   user_name      3 non-null      object             
 4   game_id        3 non-null      int64              
 5   game_name      3 non-null      object             
 6   type           3 non-null      object             
 7   title          3 non-null      object             
 8   viewer_count   3 non-null      int64              
 9   started_at     3 non-null      datetime64[ns, UTC]
 10  language       3 non-null      object             
 11  thumbnail_url  3 non-null      object             
 12  tag_ids        3 non-null      object             
 13  is_mature      3 non-null      bool               
dt

In [74]:
temp

,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,is_mature
9,42697432524,15564828,nickmercs,NICKMERCS,33214,Fortnite,live,Fraggin' | Code: MFAM | @NICKMERCS,29598,2021-07-20 15:18:41+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,['6ea6bca4-4712-4ab9-a906-e3336a9d8039'],False
16,42697924396,82524912,bugha,Bugha,33214,Fortnite,live,NEW BUGHA SKIN & GAMEMODE w/ FAZE SWAY & ZYFA,23941,2021-07-20 16:09:34+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"['24cc3843-e288-43f9-9c35-50fe1571c341', '6ea6...",False
62,42696784428,416689874,fazesway,FaZeSway,33214,Fortnite,live,Bugha's Late Game W/ Bugha and Zyfa | !vid !sac,8101,2021-07-20 14:08:52+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,['6ea6bca4-4712-4ab9-a906-e3336a9d8039'],False


In [76]:
temp_total_viewers = temp.viewer_count.sum()
temp_total_viewers

61640

In [77]:
temp_total_streamers = temp.user_login.nunique()
temp_total_streamers

3

In [86]:
largedf = pd.DataFrame(list(db.continuous_games.find({})))

In [88]:
largedf.started_at = pd.to_datetime(largedf.started_at)

In [89]:
largedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25349 entries, 0 to 25348
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   _id            25349 non-null  object             
 1   id             25349 non-null  object             
 2   user_id        25349 non-null  object             
 3   user_login     25349 non-null  object             
 4   user_name      25349 non-null  object             
 5   game_id        25349 non-null  object             
 6   game_name      25349 non-null  object             
 7   type           25349 non-null  object             
 8   title          25349 non-null  object             
 9   viewer_count   25349 non-null  int64              
 10  started_at     25349 non-null  datetime64[ns, UTC]
 11  language       25349 non-null  object             
 12  thumbnail_url  25349 non-null  object             
 13  tag_ids        25322 non-null  object         

In [98]:
def game_info(game):
    df = pd.DataFrame(list(db.continuous_games.find({'game_name':game})))
    viewers = df.viewer_count.sum()
    streamers = df.user_login.nunique()
    print('The total number of viewers is: {}\n The total number of streamers is {}'.format(viewers, streamers))
    return viewers, streamers


In [99]:
game_info('Fortnite')

The total number of viewers is: 349306
 The total number of streamers is 188


(349306, 188)

In [103]:
largedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25349 entries, 0 to 25348
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   _id            25349 non-null  object             
 1   id             25349 non-null  object             
 2   user_id        25349 non-null  object             
 3   user_login     25349 non-null  object             
 4   user_name      25349 non-null  object             
 5   game_id        25349 non-null  object             
 6   game_name      25349 non-null  object             
 7   type           25349 non-null  object             
 8   title          25349 non-null  object             
 9   viewer_count   25349 non-null  int64              
 10  started_at     25349 non-null  datetime64[ns, UTC]
 11  language       25349 non-null  object             
 12  thumbnail_url  25349 non-null  object             
 13  tag_ids        25322 non-null  object         

In [111]:
largedf.groupby(['game_name','started_at'])[['viewer_count']].sum()

AttributeError: 'str' object has no attribute 'dt'

In [114]:
largedf['hour'] = largedf.started_at.dt.hour

In [115]:
largedf

,_id,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,is_mature,hour
0,60f9f87372c271762272d611,42946210749,207813352,hasanabi,HasanAbi,509658,Just Chatting,live,TW// MURDER REACTS W/ VALKYRAE !duo,43709,2021-07-22 18:08:27+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[6ea6bca4-4712-4ab9-a906-e3336a9d8039, 77017b7...",False,18
1,60f9f87372c271762272d612,42948510365,26301881,sodapoppin,sodapoppin,509658,Just Chatting,live,Driving steering wheel/Genshin tonight | Merch...,26969,2021-07-22 21:17:04+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[6ea6bca4-4712-4ab9-a906-e3336a9d8039, c1ea449...",True,21
2,60f9f87372c271762272d613,42719076284,137347549,locklear,Locklear,509658,Just Chatting,live,REAC NOUVEAU BATTLEFIELD 2042,26628,2021-07-22 18:13:17+00:00,fr,https://static-cdn.jtvnw.net/previews-ttv/live...,[6f655045-9989-4ef7-8f85-1edcec42d648],False,18
3,60f9f87372c271762272d614,42947715197,62438432,h3h3productions,h3h3productions,509658,Just Chatting,live,H3TV | Amouranth Interview - DRAMA - David Dob...,24523,2021-07-22 20:15:16+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],False,20
4,60f9f87372c271762272d615,39695840747,125387632,amouranth,Amouranth,509658,Just Chatting,live,BAN APPEALS❌ !s --> all my social media❌!insta...,10418,2021-07-22 18:39:13+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],False,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25344,60fa14a50ce3259904e8160b,42722559532,66507380,valicvacosties,ValicVacosties,509664,Tabletop RPGs,live,Dungeons and Dragons with Valic GM'ing for the...,1,2021-07-23 00:17:31+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,"[06ab147d-91b6-4c7f-83e4-2b337d2780d5, 6ea6bca...",True,0
25345,60fa14a50ce3259904e8160c,42950819181,410179402,thegodfoca,TheGodFoca,509664,Tabletop RPGs,live,Rpgzinho! Vampiro a Máscara - Aventura em LA,1,2021-07-23 00:07:05+00:00,pt,https://static-cdn.jtvnw.net/previews-ttv/live...,[39ee8140-901a-4762-bfca-8260dea1310f],False,0
25346,60fa14a50ce3259904e8160d,42951417533,42545958,huyneo,Huyneo,509664,Tabletop RPGs,live,Shadowrun PowerPoint Creation - Terrible Terrible,1,2021-07-23 00:48:27+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],False,0
25347,60fa14a50ce3259904e8160e,42949876877,571113604,whencanweplay,WhenCanWePlay,509664,Tabletop RPGs,live,Blades in The Dark Episode 9 | WhenCanWePlay |...,1,2021-07-22 23:02:00+00:00,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[6ea6bca4-4712-4ab9-a906-e3336a9d8039],True,23


In [120]:
largedf.groupby(['game_name','hour'])[['viewer_count']].sum()

viewer_count
game_name       hour              
A Township Tale 15               3
                18            5864
                21             128
                23               9
ASMR            0              451
...                            ...
osu!            19            3460
                20             282
                21            1535
                22             687
                23             397

[1387 rows x 1 columns]

In [123]:
fortdf = largedf[largedf.game_name == 'Fortnite']

In [125]:
fortdf.groupby(['hour'])[['viewer_count']].sum()

,viewer_count
hour,
0,1797
9,336
11,345
14,1008
15,432
16,2323
17,3176
18,11906
19,27292
